In [1]:
# 순차적인 신경망을 구성할 때 사용할 수 있는 함수가 케라스의 모델 도구(models) 중 시퀀셜 모델(Sequential)함수
from tensorflow.keras.models import Sequential

# 레이어 도구(layers) 중 Dense와 Activation 도구를 불러오는 명령어.
# Dense는 전결합층(fully connected layer)를 의미함. 
# 입력측, 은닉층, 출력층과 같은 각각의 층들이 바로 앞의 층과 연결되어 있는 것을 전결합층이라고 한다.
from tensorflow.keras.layers import Dense, Activation

# 유틸 도구(utils) 중 to_categorical 함수를 불러오는 명령어
# 우리가 만들 인공지능 모델은 0부터 9사이에 있는 숫자 이미지를 구별하는 AI이다. (-> 다중분류 )
# 이때 이미지를 잘 학습하기 위한 방법 중 하나로 원-핫 인코딩을 사용한다.
# 이 원-핫 인코딩을 구현할 수 있는 함수가 to_categorical 함수이다.
from tensorflow.keras.utils import to_categorical

# 케라스를 사용해 딥러닝 모델 개발을 연습할 수 잇는 여러 데이터셋을 제공함 그중 mnist 데이터셋을 불러옴
from tensorflow.keras.datasets import mnist

import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
# load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train 등은 모두 numpy 배열(ndarray)
print("x_train shape : ", x_train.shape) # 6만개의 값, 28x28 
print("y_train shape : ", y_train.shape) # 정답 1차원 6만개
print("x_test shape : ", x_test.shape)
print("y_test shape : ", y_test.shape)

In [ ]:
# 데이터 전처리 시작!

#  mnist 데이터셋에서 X의 형태 바꾸기
# 28x28 데이터를 1x784 형태처럼 한 줄로 만든 후 이를 딥러닝 모델에 입력
X_train = x_train.reshape(60000, -1)
X_test = x_test.reshape(10000, -1)

# 정규화를 위해 데이터를 0~1 사이의 값으로 바꾸려고함
# 이 값들은 모두 실수인데 현재 X_train 데이터는 정수형이므로 자료형을 바꿔줘야함
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# mnist 데이터의 각형태는 검은색은 0, 흰색은 255, 회색은 1-254 사이의 값이다.
# 정규화를 위해 /255를 함 
X_train /=255
X_test /=255

print("X Training matrix shape : ", X_train.shape)
print("X Test matrix shape : ", X_test.shape)

In [ ]:
# mnist 데이터세에서 Y의 형태 바꾸기
# 현재 만드는 AI의 목표 : 숫자의 특성보다는 '구분'을 잘 하는 것!
# 이미지의 레이블(정답)을 더 잘 구분할 수 있는 방법으로 알려줄 필요가 있다.
# 이를 수치형 데이터 범주형 데이터로 변환한다고 한다.

# 예측이 아닌 분류 문제에선 대부분 정답 레이블을 첫 번째, 두 번째, 세 번째와 같이 순서로 나타내도록 데이터의 형태를 바꾼다.
# 이때 원-핫 인코딩(one-hot incoding)을 사용.

# to_categorical 함수는 수치형 데이털르 범주형 데이터로 만들어주는 함수이다.
# Ai가 예측하는 결과는 0~9까지의 숫자이므로 분류하고자 하는 값은 10개이다. 따라서 원핫인코딩으로 구분하련느 수를 10으로 설정한다.
Y_train = to_categorical(y_train, 10)
Y_test = to_categorical(y_test, 10)

print("Y Training matrix shape : ", Y_train.shape)
print("Y Testing matrix shape : ", Y_test.shape)

# 데이터 전처리 끝!


In [ ]:
# Ai 모델 설계하기
# 우리가 설계하고 있는 인공지능 모델은 4개의 층으로 이루어짐
# 첫 번째 층 : 입력층, 2,3번째 : 은닉층, 4번째 : 출력층

# 입력층 784, 은닉층1 512, 은닉층2 256, 출력층 10
# 우리가 넣는 데이터의 모습이 28x28을 784의 1줄로 이루어졌기 때문에 입력층의 뉴런 수는 784개
# 첫 번째 은닉층의 노드는 512개, 1->2로 갈 때 '렐루함수'사용

# 두 번째 은닉층의 노드는 256개 (이건 임의로 설정)
# 여기서 마지막 층으로 갈 때에도 활성화 함수는 렐루 함수를 사용
# 마지막 노드가 10개인 이윤느 입력된 이미지 10개를 구분하기 위함
# 그리고 가장 높은 확률의 값으로 분류하기 위해 각 노드의 최종 값을 소프트맥스 함수를 사용해 나타냄

model = Sequential()
model.add(Dense(512, input_shape = (784,)))
model.add(Activation('relu'))
model.add(Dense(256)) # 두 번째 은닉층부턴 입력받는 노드를 설정해 줄 필요가 없다. (이것이 케라스를 사용하는 이유)
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

# Layer를 나타네는 Layer 부분 / 레이어의 모습을 나타내는 Output Shape 부분 / 각 노드와 편향을 연결하는 가중치의 수를 나타내는 param부분
# 첫 번째 레이어는 512개의 노드로 이루어져 있고, 총 401,920(784*512 + 512)개 파라미터로 이루어져있다.

In [ ]:
# 모델 학습시키기

# 신경망을 잘 학습시키려면 학습한 신경망이 분류한 값과 싲레 값의 오차부터 계산해야 한다.
# 그리고 오차를 줄이기 위해 경사하강법을 사용한다.

# 케라스는 심층 신경망의 학습하는 방법을 정하는 명령어인 compile 함수 제공
# compile 함수 사용 규칙 1), 2), 3)
# 1) 오차값을 계산한느 방법을 알려줘야함 : 현재는 다중분류 문제이므로 categorical_crossentropy 방법 
# 2) 오차를 줄이는 방법을 알려줘야 한다 : 옵티마이저(optimizer)를 사용 -> 경사하강법이 여러개 있는데 이 중 하나가 adam이라는 방법이다.
# 3) 학습결과를 어떻게 학습할지 알려줘야한다 : 여기선 정확도로 모델 학습 결과를 확인
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=['accuracy'])

# 케라스는 학습을 위해 맞춘다는 의미를 가진 fit 함수 제공
# 1) 입력할 데이터를 정한다.
# 2) batch size를 정한다 : batch 사이즈는 인공지능 모델이 한 번에 학습하는 데이터의 수
# 3) epochs를 정한다. epochs는 모든 데이털르 한 번 학습하는 것을 의미, 여기선 10번 반복해서 학습시킨다.
# 에포큰느 10, verbose는 1로 한다. verbos는 케라스 fit함수의 결괏값을 출력하는 방법이고 그 값은 0,1,2중 하나로 결정
# verbose를 0으로 하면 아무 표시X, 1은 에코프별 진행사항을 알려줌 2는 결과를 알려줌
model.fit(X_train, Y_train, batch_size = 128, epochs= 10, verbose = 1)

# 결괄르 보면 처음부터 10번까지 epochs로 갈 수록 오차값은 줄고 정확도는 올라감
# 6만개를 128개로 나눠서 하면 468개. 

In [ ]:
# 모델 학습 결과 확인하기
# 인공지능이 잘 구분한 그림과 잘 구분하지 않은 그림 살펴보기

# 테스트 데이터를 예측(predict)하고 거기서 가장 큰 값이 몇 번째에 있는 지를 받아옴
# 이때 axis = 1로 가로 중에서 가장 큰 값을 출력함
# predicted_classes는 배열이 될 것이다.
predicted_classes = np.argmax(model.predict(X_test), axis = 1)

# 맞은 값은 예측한 값을 담은 것과 실제 정답이 동일하면 true의미로 1이나오고
# nonzeoro 함수는 0이 아닌 값이 '어디' 있는지를 찾아내 그 위치를 correct_indeces에 넣어줌
correct_indices = np.nonzero(predicted_classes== y_test)[0]

# 틀린 값은 false의미로 0이 나온다.
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

In [ ]:
# 잘 에측한 데이터 살펴보기
# matplotlib 라이브러리로 화면에 그래프 출력하기

plt.figure() # 그래프를 그리겠다는 준비하는 명령어
for i in range(9):
    # subplot은 그림의 위치를 정해주는 함수 (가로개수, 세로개수, 순서)
    plt.subplot(3, 3, i+ 1)
    correct = correct_indices[i]
    
    # X_test 변수에 있는 첫 번째 그림(correct 변수에는 첫 번째 그림을 의미하는 0이 들어있으므로)
    # 앞서서 데이터를 1행만 가지게 쭉 펴주었기때문에 reshpae로 28x28로 만들어야함 
    plt.imshow(X_test[correct].reshape(28,28), cmap = 'gray')

    plt.title("Predicted{}, Class {}".format(predicted_classes[correct], y_test[correct]))
    
plt.tight_layout() # for문을 나와, 화면에 그림을 보여주기 위한 함수



In [ ]:
# 잘 에측하지 못한 데이터 살펴보기
# matplotlib 라이브러리로 화면에 그래프 출력하기

plt.figure() # 그래프를 그리겠다는 준비하는 명령어
for i in range(9):
    # subplot은 그림의 위치를 정해주는 함수 (가로개수, 세로개수, 순서)
    plt.subplot(3, 3, i+ 1)
    incorrect = incorrect_indices[i]
    
    # X_test 변수에 있는 첫 번째 그림(correct 변수에는 첫 번째 그림을 의미하는 0이 들어있으므로)
    # 앞서서 데이터를 1행만 가지게 쭉 펴주었기때문에 reshpae로 28x28로 만들어야함 
    plt.imshow(X_test[incorrect].reshape(28,28), cmap = 'gray')

    plt.title("Predicted{}, Class {}".format(predicted_classes[incorrect], y_test[incorrect]))
    
plt.tight_layout() # for문을 나와, 화면에 그림을 보여주기 위한 함수

